<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Multi-Query-Retriever

This method can automatically use a ChatModel to make slight variations of your initial query to help attempt overcome any issues with cosine similarity distances, this allows you to phrase things more like a general query question rather than a pure document similarity look-up.

In [1]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

### Loader

In [4]:
loader = WikipediaLoader(query='MKUltra')
documents = loader.load()

/home/ankdesh/virtualenv/pytorch-host/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/ankdesh/virtualenv/pytorch-host/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [5]:
len(documents)

23

### Split Documents

In [6]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

Created a chunk of size 516, which is longer than the specified 500


### OpenAI Connection for Embeddings

In [7]:
import os
embedding_function = OpenAIEmbeddings()

In [8]:
# docs

### Embed Documents for ChromaDB

In [9]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function,persist_directory='./mk_ultra')
db.persist()

### Use Chat Model to Multi Query

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
question="When was this declassified?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(),llm=llm)

In [11]:
# Set logging for the queries
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [12]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of declassification for this information?', '2. Can you provide the declassification date for this?', '3. At what time was this information officially declassified?']


In [13]:
len(unique_docs)

6

In [14]:
print(unique_docs[0].page_content)

The Church Committee (formally the United States Senate Select Committee to Study Governmental Operations with Respect to Intelligence Activities) was a US Senate select committee in 1975 that  investigated abuses by the Central Intelligence Agency (CIA), National Security Agency (NSA), Federal Bureau of Investigation (FBI), and the Internal Revenue Service (IRS). Chaired by Idaho Senator Frank Church (D-ID), the committee was part of a series of investigations into intelligence abuses in 1975, dubbed the "Year of Intelligence", including its House counterpart, the Pike Committee, and the presidential Rockefeller Commission. The committee's efforts led to the establishment of the permanent US Senate Select Committee on Intelligence.
The most shocking revelations of the committee include Operation MKULTRA, which involved the drugging and torture of unwitting US citizens as part of human experimentation on mind control; COINTELPRO, which involved the surveillance and infiltration of Amer

In [20]:
print (unique_docs[0])

page_content='The Church Committee (formally the United States Senate Select Committee to Study Governmental Operations with Respect to Intelligence Activities) was a US Senate select committee in 1975 that  investigated abuses by the Central Intelligence Agency (CIA), National Security Agency (NSA), Federal Bureau of Investigation (FBI), and the Internal Revenue Service (IRS). Chaired by Idaho Senator Frank Church (D-ID), the committee was part of a series of investigations into intelligence abuses in 1975, dubbed the "Year of Intelligence", including its House counterpart, the Pike Committee, and the presidential Rockefeller Commission. The committee\'s efforts led to the establishment of the permanent US Senate Select Committee on Intelligence.\nThe most shocking revelations of the committee include Operation MKULTRA, which involved the drugging and torture of unwitting US citizens as part of human experimentation on mind control; COINTELPRO, which involved the surveillance and infi